# GAN tutorial

## 1, Create Generator and Discriminator

In [2]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import transforms

In [2]:
# Discriminator
class discriminator(nn.Module):
    def __init__(self):
        super(discriminator, self).__init__()
        self.fc1 = nn.Linear(784, 512) # (input layer, hidden layer)
        self.fc2 = nn.Linear(512, 1) # (hidden layer, output layer)
        self.activation = nn.LeakyReLU(0.1)

    def forward(self, x):
        x = x.view(-1, 784)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return nn.Sigmoid()(x)

# Generator
class generator(nn.Module):
    def __init__(self):
        super(generator, self).__init__()
        self.fc1 = nn.Linear(128, 1024)
        self.fc2 = nn.Linear(1024, 2048)
        self.fc3 = nn.Linear(2048, 784)
        self.activation = nn.ReLU()

    def forward(self, x):
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.fc3(x)
        x = x.view(-1, 1, 28, 28)
        return nn.Tanh()(x)

## 2, Training (Using GPU) 

-----ここからは家で帰ってから実装すること-----

In [1]:
import numpy as np
import matplotlib.pyplot as plt

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


# Setting hyper parameter
epochs = 30  # 学習回数
lr = 2e-4  # 学習率
batch_size = 64  # データを細かくしたもの
loss = nn.BCELoss()

# Create model
G = generator.to(device)
D = generator.to(device)

G_optimizer = optim.Adam(G.parameters(), lr=lr, betas=(0.5, 0, 999))
D_optimizer = optim.Adam(d.parameters(), lr=lr, betas=(0.5, 0.999))

# Transform
transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,),(0.5,))])

train_set = datasets.MNIST('mnist/', train=True, download=True, transform=transform)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)

# Start Training
for epoch in range(epochs):
    for idx, (imgs, _) in enumerate(train_loader):
        idx += 1

        # training discriminator
        real_inputs = imgs.to(device)
        real_outputs = D(real_inputs)
        real_label = torch.ones(real_inputs.shape[0], 1).to(device)

        noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
        noise = noise.to(device)
        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_label = torch.zeros(fake_inputs.shape[0], 1).to(device)

        outputs = torch.cat((real_outputs, fake_outputs), 0)
        targets = torch.cat((real_label, fake_label), 0)

        D_loss = loss(outputs, targets)
        D_optimizer.zero_grad()
        D_loss.backward()
        D_optimizer.step()

        noise = (torch.rand(real_inputs.shape[0], 128-0.5) / 0.50)
        noise = noise.to(device)

        fake_inputs = G(noise)
        fake_outputs = D(fake_inputs)
        fake_targets = torch.ones([fake_inputs.shape[0], 1]).to(device)
        G_loss = loss(fake_outputs, fake_targets)
        G_optimizer.zero_grad()
        G_loss.backward()
        G_optimizer.step()

        if idx % 100 == 0 or idx == len(train_loader):
            print(f'Epoch {epoch} Iteration{idx}: discriminator_loss {D_loss.item()} generator_loss{G_loss.item()}')

    if (epoch+1) % 10 == 0:
        torch.save(G, f'Generator_epoch_{epoch}.pth')
        print('Model saved.')



NameError: name 'torch' is not defined

## 3, Create Image using GAN

In [ ]:
for i in input:
    print("real")
    plt.imshow(i[0][0].reshape(28, 28))
    plt.show()
    real_inputs = i[0][0]
    noise = (torch.rand(real_inputs.shape[0], 128) - 0.5) / 0.5
    noise = noise.to(device)
    fake_inputs = G(noise)
    print("fake")
    plt.imshow(fake_inputs[0][0].cpu().detach().numpy().reshape(28, 28))
    plt.show()
    break
